# Homework 2

## FINM 35700 - Spring 2023

### UChicago Financial Mathematics

* Aman Krishna
* amank@uchicago.edu

This homework relies on:

- the US government bonds symbology file `govt_symbology`, 
- the "on-the-run" treasuries data file `govt_on_the_run`,
- the corporate  bonds symbology file `corp_symbology` and
- the market data file `market_prices_eod`.


# Problem 1: Constructing fixed rate bonds

In [111]:
import QuantLib as ql
import numpy as np
import pandas as pd
import datetime as dt
import os

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Use static calculation/valuation date of 2023-04-14, matching data available in the market prices EOD file
calc_date = ql.Date(14, 4, 2023)
ql.Settings.instance().evaluationDate = calc_date

mydir = os.getcwd()

In [34]:
# Calculate initial term and current time-to-maturity for each bond issue
def get_symbology(df, underlying=False):
    for index, row in df.iterrows():
        start_date = ql.Date(row['start_date'].day, row['start_date'].month, row['start_date'].year)
        maturity_date = ql.Date(row['maturity'].day, row['maturity'].month, row['maturity'].year)
        today_date = ql.Date(14,4,2023)
        calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
        #set dcc as Actual/365.25
        dcc = ql.Actual36525()
        initial_term = dcc.yearFraction(start_date, maturity_date)
        current_time_to_maturity = dcc.yearFraction(today_date, maturity_date)
        df.at[index, 'term'] = initial_term
        df.at[index, 'TTM'] = current_time_to_maturity

    if underlying:
        return df[['ticker','figi','security','name','und_bench_isin','coupon','days_settle','start_date','acc_first', 'maturity', 'term', 'TTM']]
    else:
        #round term to integer
        df['term'] = df['term'].apply(np.ceil)
        df1 = df[['ticker','figi','security','name','coupon','days_settle','start_date','acc_first', 'maturity', 'term', 'TTM']]
        return df1

## a. Prepare common symbology dataframe for fixed rate government and corporate bonds

Load the `govt_symbology` and `corp_symbology` Excel files into one common dataframe, using the `corp_symbology` columns. Filter the dataframe and keep fixed rate bonds only: (cpn_type="FIXED").


In [115]:
govt_symbology = pd.read_excel(os.path.join(mydir, 'data', 'govt_symbology.xlsx'))
#keep only cpn_type="FIXED" bonds
govt_symbology = govt_symbology[govt_symbology['cpn_type'] == 'FIXED']
#reset index
govt_symbology = govt_symbology.reset_index(drop=True)
govt_symbology.head(2)

,ticker,class,figi,isin,trace,security,name,type,coupon,cpn_type,dcc,cpn_freq,days_settle,start_date,cpn_first,acc_first,maturity,country,currency
0,T,Govt,BBG000DLJF04,US912810EQ77,TSRYS4493810,T 6 1/4 08/15/23,US TREASURY N/B,US GOVERNMENT,6.25,FIXED,ACT/ACT,2.0,1,1993-08-16,1994-02-15,1993-08-15,2023-08-15,US,USD
1,T,Govt,BBG000DKZWK9,US912810ES34,TSRYS4493811,T 7 1/2 11/15/24,US TREASURY N/B,US GOVERNMENT,7.50,FIXED,ACT/ACT,2.0,1,1994-08-15,1994-11-15,1994-05-15,2024-11-15,US,USD


In [36]:
gov_sym = get_symbology(govt_symbology)
gov_sym

,ticker,figi,security,name,coupon,days_settle,start_date,acc_first,maturity,term,TTM
0,T,BBG000DLJF04,T 6 1/4 08/15/23,US TREASURY N/B,6.250,1,1993-08-16,1993-08-15,2023-08-15,30.0,0.336756
1,T,BBG000DKZWK9,T 7 1/2 11/15/24,US TREASURY N/B,7.500,1,1994-08-15,1994-05-15,2024-11-15,31.0,1.590691
2,T,BBG000DLBVY0,T 7 5/8 02/15/25,US TREASURY N/B,7.625,1,1995-02-15,1995-02-15,2025-02-15,31.0,1.842574
3,T,BBG000DQQNJ8,T 6 7/8 08/15/25,US TREASURY N/B,6.875,1,1995-08-15,1995-08-15,2025-08-15,31.0,2.338125
4,T,BBG000DPXMD0,T 6 02/15/26,US TREASURY N/B,6.000,1,1996-02-15,1996-02-15,2026-02-15,31.0,2.841889
...,...,...,...,...,...,...,...,...,...,...,...
327,T,BBG01FQJ5MY4,T 4 5/8 03/15/26,US TREASURY N/B,4.625,1,2023-03-15,2023-03-15,2026-03-15,4.0,2.918549
328,T,BBG01G14BV75,T 3 5/8 03/31/30,US TREASURY N/B,3.625,1,2023-03-31,2023-03-31,2030-03-31,8.0,6.962355
329,T,BBG01G14BW91,T 3 5/8 03/31/28,US TREASURY N/B,3.625,1,2023-03-31,2023-03-31,2028-03-31,6.0,4.963723
330,T,BBG01G14BXX2,T 3 7/8 03/31/25,US TREASURY N/B,3.875,1,2023-03-31,2023-03-31,2025-03-31,3.0,1.963039


In [117]:
corp_symbology = pd.read_excel(os.path.join(mydir, 'data', 'corp_symbology.xlsx'))
#keep only cpn_type="FIXED" bonds
corp_symbology = corp_symbology[corp_symbology['cpn_type'] == 'FIXED']
#reset index
corp_symbology = corp_symbology.reset_index(drop=True)
corp_symbology.head(2)

,ticker,class,figi,isin,trace,und_bench_isin,security,name,type,coupon,...,cpn_freq,days_settle,start_date,cpn_first,acc_first,maturity,mty_typ,rank,country,currency
0,DIS,Corp,BBG00QNKJ092,US254687CZ75,DIS4921182,US91282CGU99,DIS 3.7 09/15/24,WALT DISNEY COMPANY/THE,GLOBAL,3.7,...,2,2,2019-11-22,2020-03-15,2019-09-15,2024-09-15,CALLABLE,Sr Unsecured,US,USD
1,DIS,Corp,BBG00QNKPCL5,US254687DD54,DIS4907444,US91282CGU99,DIS 3.7 10/15/25,WALT DISNEY COMPANY/THE,GLOBAL,3.7,...,2,2,2019-11-22,2020-04-15,2019-10-15,2025-10-15,CALLABLE,Sr Unsecured,US,USD


In [118]:
corp_sym = get_symbology(corp_symbology, underlying=True)
corp_sym

,ticker,figi,security,name,und_bench_isin,coupon,days_settle,start_date,acc_first,maturity,term,TTM
0,DIS,BBG00QNKJ092,DIS 3.7 09/15/24,WALT DISNEY COMPANY/THE,US91282CGU99,3.700,2,2019-11-22,2019-09-15,2024-09-15,4.815880,1.423682
1,DIS,BBG00QNKPCL5,DIS 3.7 10/15/25,WALT DISNEY COMPANY/THE,US91282CGU99,3.700,2,2019-11-22,2019-10-15,2025-10-15,5.897331,2.505133
2,DIS,BBG00QNKGJP6,DIS 3 3/8 11/15/26,WALT DISNEY COMPANY/THE,US91282CGR60,3.375,2,2019-11-22,2019-11-15,2026-11-15,6.981520,3.589322
3,DIS,BBG00QNKP8R8,DIS 6.55 03/15/33,WALT DISNEY COMPANY/THE,US91282CGM73,6.550,2,2019-11-22,2019-09-15,2033-03-15,13.311431,9.919233
4,DIS,BBG00QNKR4J4,DIS 6.2 12/15/34,WALT DISNEY COMPANY/THE,US91282CGM73,6.200,2,2019-11-22,2019-06-15,2034-12-15,15.063655,11.671458
...,...,...,...,...,...,...,...,...,...,...,...,...
214,VZ,BBG017BR3G75,VZ 4.15 05/15/29,VERIZON COMMUNICATIONS,US91282CGT27,4.150,2,2022-05-19,2022-05-19,2029-05-15,6.989733,6.086242
215,VZ,BBG017QYK9Z0,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,US912810TL26,4.650,2,2022-06-03,2022-06-03,2052-06-15,30.034223,29.171800
216,VZ,BBG0191HMXJ3,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,US912810TL26,4.600,2,2022-08-11,2022-08-11,2052-08-15,30.012320,29.338809
217,VZ,BBG019LXC9P3,VZ 4 1/2 09/15/29,VERIZON COMMUNICATIONS,US91282CGT27,4.500,2,2022-09-22,2022-09-22,2029-09-15,6.981520,6.422998


In [119]:
#join govt and corp symbology
symbology = pd.concat([gov_sym, corp_sym], ignore_index=True)
symbology

,ticker,figi,security,name,coupon,days_settle,start_date,acc_first,maturity,term,TTM,und_bench_isin
0,T,BBG000DLJF04,T 6 1/4 08/15/23,US TREASURY N/B,6.250,1,1993-08-16,1993-08-15,2023-08-15,30.000000,0.336756,NaN
1,T,BBG000DKZWK9,T 7 1/2 11/15/24,US TREASURY N/B,7.500,1,1994-08-15,1994-05-15,2024-11-15,31.000000,1.590691,NaN
2,T,BBG000DLBVY0,T 7 5/8 02/15/25,US TREASURY N/B,7.625,1,1995-02-15,1995-02-15,2025-02-15,31.000000,1.842574,NaN
3,T,BBG000DQQNJ8,T 6 7/8 08/15/25,US TREASURY N/B,6.875,1,1995-08-15,1995-08-15,2025-08-15,31.000000,2.338125,NaN
4,T,BBG000DPXMD0,T 6 02/15/26,US TREASURY N/B,6.000,1,1996-02-15,1996-02-15,2026-02-15,31.000000,2.841889,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
546,VZ,BBG017BR3G75,VZ 4.15 05/15/29,VERIZON COMMUNICATIONS,4.150,2,2022-05-19,2022-05-19,2029-05-15,6.989733,6.086242,US91282CGT27
547,VZ,BBG017QYK9Z0,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,4.650,2,2022-06-03,2022-06-03,2052-06-15,30.034223,29.171800,US912810TL26
548,VZ,BBG0191HMXJ3,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,4.600,2,2022-08-11,2022-08-11,2052-08-15,30.012320,29.338809,US912810TL26
549,VZ,BBG019LXC9P3,VZ 4 1/2 09/15/29,VERIZON COMMUNICATIONS,4.500,2,2022-09-22,2022-09-22,2029-09-15,6.981520,6.422998,US91282CGT27


## b. Add function to construct generic fixed rate cashflow schedules from symbology data

Use one row of the symbology dataframe as input  to the function. Use the helper function to convert a date string to a QuantLib date object.

In [120]:
def get_ql_date(date) -> ql.Date:
    """
    convert dt.date to ql.Date
    """
    if isinstance(date, dt.date):
        return ql.Date(date.day, date.month, date.year)
    elif isinstance(date, str):
        date = dt.datetime.strptime(date, "%Y-%m-%d").date()
        return ql.Date(date.day, date.month, date.year)
    else:
        raise ValueError(f"to_qldate, {type(date)}, {date}")

In [121]:
def create_schedule_from_symbology(details: dict):
    '''Create a QuantLib cashflow schedule from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    # Create maturity from details['maturity']
    maturity = ql.Date(details['maturity'].day, details['maturity'].month, details['maturity'].year)
    
    # Create acc_first from details['acc_first']
    acc_first = ql.Date(details['acc_first'].day, details['acc_first'].month, details['acc_first'].year)
    
    # Create calendar for Corp and Govt asset classes
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    
    # define period from details['cpn_freq'] ... can be hard-coded to 2 = semi-annual frequency
    period = ql.Period(ql.Semiannual)
    
    # business_day_convention
    business_day_convention = ql.Unadjusted
    
    # termination_date_convention
    termination_date_convention = ql.Unadjusted
    
    # date_generation
    date_generation=ql.DateGeneration.Backward
    
    # Create schedule using ql.MakeSchedule interface (with keyword arguments)
    schedule = ql.MakeSchedule(effectiveDate=acc_first,  # this may not be the same as the bond's start date
                            terminationDate=maturity,
                            tenor=period,
                            calendar=calendar,
                            convention=business_day_convention,
                            terminalDateConvention=termination_date_convention,
                            rule=date_generation,
                            endOfMonth=True,
                            firstDate=ql.Date(),
                            nextToLastDate=ql.Date())
    return schedule

## c. Add function to construct generic fixed rate bond objects from symbology data

Use one row of the symbology dataframe as input to the function. Use create_schedule_from_symbology() internally to create the cashflow schedule.

In [122]:
def create_bond_from_symbology(details: dict):
    '''Create a US fixed rate bond object from symbology details dictionary (usually one row of the symbology dataframe)
    '''
    
     # Create day_count from details['dcc']
     # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
     # For US Corporates use ql.Thirty360(ql.Thirty360.USA)
     
    if details['ticker'] == 'T':
        day_count = ql.ActualActual(ql.ActualActual.ISMA)
    else:
        day_count = ql.Thirty360(ql.Thirty360.USA)
    
    # Create day_count from details['start_date']    
    issue_date = ql.Date(details['start_date'].day, details['start_date'].month, details['start_date'].year)
    
    # Create days_settle from details['days_settle']
    days_settle = int(float(details['days_settle']))

    # Create days_settle from details['coupon']
    coupon = float(details['coupon'])/100.

    # Create cashflow schedule
    schedule = create_schedule_from_symbology(details)
    
    face_value = 100
    redemption = 100
    
    payment_convention = ql.Unadjusted
        
    # Create fixed rate bond object
    fixed_rate_bond = ql.FixedRateBond(
        days_settle,
        face_value,
        schedule,
        [coupon],
        day_count,
        payment_convention,
        redemption,
        issue_date)        

    return fixed_rate_bond


## d. Add function that returns a dataframe with (future) cash flows details for a bond object

Use the "Investigate Bond Cashflows" section in the Quantlib introductory notebook as a template.

The results dataframe should contain following columns:


| CashFlowDate | CashFlowAmount | CashFlowYearFrac |
|----------|-------|-------------|

Pick one government and one corporate bond from symbology, create the bond objects and display the future cashflows.

In [123]:
def get_bond_cashflows(bond: ql.FixedRateBond, calc_date=ql.Date) -> pd.DataFrame:
    '''Returns all future cashflows as of calc_date, i.e. with payment dates > calc_date.
    '''    
    # Create cashflows list based on calc_date and hasOccurred argument and yearFraction using bond.dayCounter()
    x = [(cf.date(), cf.amount(), bond.dayCounter().yearFraction(calc_date, cf.date())) for cf in bond.cashflows() if cf.hasOccurred() == False]
    
    cashflows_df = pd.DataFrame(x, columns=['CashFlowDate', 'CashFlowAmount', 'CashFlowYearFrac'])
    
    '''Below code can be used to merge the rows with same CashFlowDate and add CashFlowAmount'''
    # #merge the rows with same CashFlowDate and add CashFlowAmount
    # cashflows_df = cashflows_df.groupby(['CashFlowDate', 'CashFlowYearFrac']).sum().reset_index()
    
    # #rearrange columns
    # cashflows_df = cashflows_df[['CashFlowDate', 'CashFlowAmount','CashFlowYearFrac']]
    
    return cashflows_df

> We show the cashflows for the below bond

In [124]:
symbology.iloc[4]

ticker                              T
figi                     BBG000DPXMD0
security                 T 6 02/15/26
name                  US TREASURY N/B
coupon                            6.0
days_settle                         1
start_date        1996-02-15 00:00:00
acc_first         1996-02-15 00:00:00
maturity          2026-02-15 00:00:00
term                             31.0
TTM                          2.841889
und_bench_isin                    NaN
Name: 4, dtype: object

In [125]:
bond_tsy_1 = create_bond_from_symbology(symbology.iloc[4].to_dict())
get_bond_cashflows(bond_tsy_1, calc_date = ql.Date(14, 4, 2023))

,CashFlowDate,CashFlowAmount,CashFlowYearFrac
0,"August 15th, 2023",3.0,0.333333
1,"February 15th, 2024",3.0,0.833333
2,"August 15th, 2024",3.0,1.333333
3,"February 15th, 2025",3.0,1.833333
4,"August 15th, 2025",3.0,2.333333
5,"February 15th, 2026",3.0,2.833333
6,"February 15th, 2026",100.0,2.833333


>Next we pick the following corp bond from the symbology dataframe

In [126]:
symbology[symbology["ticker"]=="IBM"].iloc[0]

ticker                            IBM
figi                     BBG0000362Y4
security               IBM 7 10/30/25
name                         IBM CORP
coupon                            7.0
days_settle                         2
start_date        1995-10-30 00:00:00
acc_first         1995-10-30 00:00:00
maturity          2025-10-30 00:00:00
term                        30.001369
TTM                          2.546201
und_bench_isin           US91282CGU99
Name: 375, dtype: object

In [127]:
bond_corp_1 = create_bond_from_symbology(symbology[symbology["ticker"]=="IBM"].iloc[0].to_dict())
get_bond_cashflows(bond_corp_1, calc_date = ql.Date(14, 4, 2023))

,CashFlowDate,CashFlowAmount,CashFlowYearFrac
0,"April 30th, 2023",3.5,0.044444
1,"October 30th, 2023",3.5,0.544444
2,"April 30th, 2024",3.5,1.044444
3,"October 30th, 2024",3.5,1.544444
4,"April 30th, 2025",3.5,2.044444
5,"October 30th, 2025",3.5,2.544444
6,"October 30th, 2025",100.0,2.544444


# Problem 2: US Treasury yield curve calibration (On-The-Runs)

## a. Create the on-the-run US treasury bond objects

Restrict the symbology + market data dataframe to on-the-run US treasury notes only (excluding Treasury Bills) and create the treasury bond objects.

Extend the treasuries symbology dataframe with the following market data columns (code from Homework 1):


| date | bid | ask | mid | bid_yield | ask_yield | mid_yield | term | TTM |
|----------|-------|-------------|-----|----------|---------|---------|---------|---------|

In [129]:
gov_otr = pd.read_excel(os.path.join(mydir, 'data', 'govt_on_the_run.xlsx'))
gov_otr.head()

,ticker,date,isin,figi
0,GT10 Govt,2023-04-14,US91282CGM73,BBG01DVKBC66
1,GT10B Govt,2023-04-14,US91282CFV81,BBG01BC373F4
2,GT10C Govt,2023-04-14,US91282CFF32,BBG01920ZBH5
3,GT2 Govt,2023-04-14,US91282CGU99,BBG01G14BXX2
4,GT20 Govt,2023-04-14,US912810TQ13,BBG01F6N6GN1


In [131]:
market_data = pd.read_excel(os.path.join(mydir, 'data', 'market_prices_eod-1.xlsx'))
market_data.head()

,date,class,ticker,isin,figi,bid,ask,mid_clean,mid_dirty,bid_yield,ask_yield
0,2023-04-14,Corp,DIS,US254687CZ75,BBG00QNKJ092,98.678,98.848,98.763,99.102,4.679,4.551
1,2023-04-14,Corp,DIS,US254687DD54,BBG00QNKPCL5,98.454,98.821,98.637,98.668,4.361,4.203
2,2023-04-14,Corp,DIS,US254687DK97,BBG00QNKGJP6,97.090,97.369,97.230,98.664,4.260,4.174
3,2023-04-14,Corp,DIS,US254687DV52,BBG00QNKP8R8,115.163,115.984,115.574,116.174,4.624,4.529
4,2023-04-14,Corp,DIS,US254687DZ66,BBG00QNKR4J4,112.939,113.586,113.262,115.380,4.742,4.674


In [136]:
symbology_md = pd.merge(symbology, market_data, on='figi', how='inner')
#restrict to isin or und_bench_isin columns equal to one of the govt_otr isins
symbology_md = symbology_md[symbology_md['isin'].isin(gov_otr['isin']) | symbology_md['und_bench_isin'].isin(gov_otr['isin'])]
symbology_md

,ticker_x,figi,security,name,coupon,days_settle,start_date,acc_first,maturity,term,...,date,class,ticker_y,isin,bid,ask,mid_clean,mid_dirty,bid_yield,ask_yield
84,T,BBG01920Z8R1,T 3 08/15/52,US TREASURY N/B,3.000,1,2022-08-15,2022-08-15,2052-08-15,31.000000,...,2023-04-14,Govt,T,US912810TJ79,86.7188,86.7969,86.7578,87.2637,3.750,3.745
85,T,BBG01958YJ98,T 3 3/8 08/15/42,US TREASURY N/B,3.375,1,2022-08-31,2022-08-15,2042-08-15,20.000000,...,2023-04-14,Govt,T,US912810TK43,93.3594,93.4688,93.4141,93.9824,3.866,3.857
86,T,BBG01BC36V36,T 4 11/15/52,US TREASURY N/B,4.000,1,2022-11-15,2022-11-15,2052-11-15,31.000000,...,2023-04-14,Govt,T,US912810TL26,104.8750,104.9531,104.9141,106.6055,3.726,3.722
87,T,BBG01BGVXMB1,T 4 11/15/42,US TREASURY N/B,4.000,1,2022-11-30,2022-11-15,2042-11-15,20.000000,...,2023-04-14,Govt,T,US912810TM09,102.1719,102.2500,102.2109,103.9023,3.841,3.835
88,T,BBG01DVKBH56,T 3 5/8 02/15/53,US TREASURY N/B,3.625,1,2023-02-15,2023-02-15,2053-02-15,31.000000,...,2023-04-14,Govt,T,US912810TN81,97.9531,98.0313,97.9922,98.6035,3.739,3.735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,VZ,BBG017BR3G75,VZ 4.15 05/15/29,VERIZON COMMUNICATIONS,4.150,2,2022-05-19,2022-05-19,2029-05-15,6.989733,...,2023-04-14,Corp,VZ,US92346MJT99,95.2390,96.3430,95.7910,97.5550,5.070,4.852
547,VZ,BBG017QYK9Z0,VZ 4.65 06/15/52,VERIZON COMMUNICATIONS,4.650,2,2022-06-03,2022-06-03,2052-06-15,30.034223,...,2023-04-14,Corp,VZ,US92346MJZ59,86.7820,89.2070,87.9940,89.5830,5.572,5.388
548,VZ,BBG0191HMXJ3,VZ 4.6 08/15/52,VERIZON COMMUNICATIONS,4.600,2,2022-08-11,2022-08-11,2052-08-15,30.012320,...,2023-04-14,Corp,VZ,US92346MKG59,86.5280,88.9730,87.7500,88.5550,5.533,5.349
549,VZ,BBG019LXC9P3,VZ 4 1/2 09/15/29,VERIZON COMMUNICATIONS,4.500,2,2022-09-22,2022-09-22,2029-09-15,6.981520,...,2023-04-14,Corp,VZ,US92346MKQ32,97.8060,99.0740,98.4400,98.8520,4.902,4.668


## b. Calibrate the on-the-run treasury yield curve (bootstrapping)
The function below shows how to calibrate a smooth yield/discount factor curve from the on-the-run treasury dataframe. Calibrate the bid, ask and mid discount factor curves as of 2023-04-14.

In [6]:
def calibrate_yield_curve_from_frame(
        calc_date: ql.Date,
        treasury_details: pd.DataFrame,
        price_quote_column: str):
    '''Create a calibrated yield curve from a details dataframe which includes bid/ask/mid price quotes.
    '''
    ql.Settings.instance().evaluationDate = calc_date

    # Sort dataframe by maturity
    sorted_details_frame = treasury_details.sort_values(by='maturity')    
    
    # For US Treasuries use ql.ActualActual(ql.ActualActual.ISMA)
    day_count = ql.ActualActual(ql.ActualActual.ISMA)

    bond_helpers = []
    
    for index, row in sorted_details_frame.iterrows():
        bond_object = create_bond_from_symbology(row)
        
        tsy_clean_price_quote = row[price_quote_column]
        tsy_clean_price_handle = ql.QuoteHandle(ql.SimpleQuote(tsy_clean_price_quote))
        
        bond_helper = ql.BondHelper(tsy_clean_price_handle, bond_object)
        bond_helpers.append(bond_helper)
        
    yield_curve = ql.PiecewiseLogCubicDiscount(calc_date, bond_helpers, day_count)
    yield_curve.enableExtrapolation()
    return yield_curve

## c. Plot the calibrated US Treasury yield (zero rate) curves

Create a graph/scatter plot of the newly computed mid yields by maturity.

## d. Plot calibrated discount factors

Plot the discount factor curve up to the 30 years point, using a 6 months discretization grid.

# Problem 3: Pricing and risk metrics for US Treasury bonds

## a. US Treasury pricing on the calibrated discount factor curve

Follow Section 5. "Present Value Calculation (no credit risk)" in the QuantLib introductory notebook to re-price the US on-the-run treasuries using the calibrated discount factor curve. 

You will need to switch the bond_engine to use the new on-the-run treasury yield curve:
bond_engine = ql.DiscountingBondEngine(treasury_yield_curve_mid_handle)

Extend the dataframe with the following computed columns for clean mid prices:


| calc_mid |
|---------------|


To validate the calibration, compare the calculated clean mid prices to the original market mid prices.

## b. Compute analytical DV01, Duration and Convexity for US on-the-run treasuries (using flat yield)

Compute analytical DV01, Duration and Convexity metrics, as described in Section 9. "Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib introductory notebook.

Remember that DV01 = Dirty_Price * Duration. 

Extend the dataframe with the following calculated risk metrics:


| dv01 | duration | convexity |
|-------|-------|-------------|


## c. Compute scenario DV01, Duration and Convexity for US on-the-run treasuries (using calibrated yield curve)

Compute the scenario DV01, Duration and Convexity metrics using +/-1bp interest rate shocks, as described in Section 6. "Market Data Scenarios" in the QuantLib introductory notebook.

Remember that DV01 = Dirty_Price * Duration.

Extend the dataframe with the following scenario sensitivities metrics:

| scen_dv01 | scen_duration | scen_convexity |
|-------|-------|-------------|



# Problem 4: Pricing and risk metrics for corporate bonds

## a. Create the on-the-run corporate bond objects

Restrict the symbology dataframe to fixed rate corporate bonds only and create the corporate bond objects.

## b. Compute analytical Yields and Z-Spreads

Compute analytical Yields and Z-Spreads metrics, as described in Section 9. "Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib introductory notebook.

Extend the dataframe with the following calculated risk metrics:


| calc_yield | calc_zspread |
|-------|-------------|


## c. Validate Z-Spread computation for a few fixed rate corporate bonds

Pick 3 corporate bonds (at your discretion) and use function below to re-price them using the calibrated flat z-spread. Follow the example in Section 9. "Duration, Convexity and Z-Spread (flat yield model)".

Validate that you match the original market price, which were used as input to the z-Spread function.


In [7]:
def calc_clean_price_with_zspread(fixed_rate_bond, yield_curve_handle, zspread):
    zspread_quote = ql.SimpleQuote(zspread)
    zspread_quote_handle = ql.QuoteHandle(zspread_quote)
    yield_curve_bumped = ql.ZeroSpreadedTermStructure(yield_curve_handle, zspread_quote_handle, ql.Compounded, ql.Semiannual)
    yield_curve_bumped_handle = ql.YieldTermStructureHandle(yield_curve_bumped)
    
    # Set Valuation engine
    bond_engine = ql.DiscountingBondEngine(yield_curve_bumped_handle)
    fixed_rate_bond.setPricingEngine(bond_engine)
    bond_clean_price = fixed_rate_bond.cleanPrice()
    return bond_clean_price


## d. Compute Duration and Convexity for fixed rate corporate bonds (using flat yield)

Compute analytical Duration and Convexity metrics, as described in Section 9. "Duration, Convexity and Z-Spread (flat yield model)" in the QuantLib introductory notebook.

Extend the dataframe with the following calculated risk metrics:


| calc_duration | calc_convexity |
|-------|-------------|
